In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from ipyparallel import Client

In [0]:
cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/"

In [0]:
mapped = !ls *mapped.bam

In [0]:
samtools = "/home/cfriedline/bin/samtools"

In [0]:
from subprocess import Popen, PIPE, STDOUT

In [0]:
rc = Client(profile = "sge")
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    from subprocess import Popen, PIPE, STDOUT

In [0]:
def get_contigs(bam):
    contigs = set()
    cmd = "samtools view %s" % bam
    p = Popen(cmd, stdout=PIPE, shell=True)
    for line in p.stdout:
        d = line.decode().split("\t")
        contigs.add(d[2])
    return contigs

In [0]:
dv.push({'samtools': samtools,
        'get_contigs': get_contigs}).r;

In [0]:
jobs = []
for b in mapped:
    jobs.append(lv.apply_async(get_contigs, b))

In [0]:
np.sum([x.ready() for x in jobs])

In [0]:
all_contigs = set()
for j in jobs:
    for contig in j.r:
        all_contigs.add(contig)

In [0]:
assembly = "/home/cfriedline/eckertlab/SugarPine_genome/pila.v1.0.scafSeq"

In [0]:
count = !grep -c ">" $assembly

In [0]:
count = int(count[0])

In [0]:
len(all_contigs), len(all_contigs)/count